In [52]:
import typing
import math
import random

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch import Tensor

In [53]:
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Current device:", torch.cuda.current_device() if torch.cuda.is_available() else "N/A")
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "N/A")

Torch version: 2.7.0+cu126
CUDA available: False
CUDA version: 12.6
Current device: N/A
Device name: N/A


In [54]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Using CUDA')
else:
    device = torch.device('cpu')
    print('Using CPU')

Using CPU


In [55]:
torch.manual_seed(1337)

In [56]:
with open('tiny-shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [57]:
len(text)

1115393

In [58]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [59]:
chars = sorted(list(set(text)))
print(str().join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [60]:
vocab_size = len(chars)
vocab_size

65

In [61]:
stoi = {ch: i for i, ch in enumerate(chars)}
print(stoi)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [62]:
itos = {i: ch for ch, i in stoi.items()}
print(itos)

{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}


In [63]:
def encode(s: str) -> typing.List[int]:
    return [stoi[c] for c in s]

def decode(ints: typing.List[int]) -> str:
    return str().join(itos[i] for i in ints)

In [64]:
print(encode('hello world'))
print(decode(encode('hello world')))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [65]:
encoded_text = encode(text)
len(encoded_text)

1115393

In [66]:
data = torch.tensor(encoded_text, dtype=torch.long, device=device)
data.shape

torch.Size([1115393])

In [67]:
data.dtype

torch.int64

In [68]:
data[:1000]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [69]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
print(f'Training size: {len(train_data)}, Validation size: {len(val_data)}')

Training size: 1003853, Validation size: 111540


In [70]:
block_size = 8 # Also called "context length"

In [71]:
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [72]:
xb = train_data[:block_size]
yb = train_data[1:block_size+1]
print('--- As characters ---')
for t in range(block_size):
    context = xb[:t+1]
    target = yb[t]
    print(f'When the input is {decode(context.tolist())} the next character is {itos[target.item()]}')
print('--- Encoded ---')
for t in range(block_size):
    context = xb[:t+1]
    target = yb[t]
    print(f'When the input is {context} the next character is {target}')

--- As characters ---
When the input is F the next character is i
When the input is Fi the next character is r
When the input is Fir the next character is s
When the input is Firs the next character is t
When the input is First the next character is  
When the input is First  the next character is C
When the input is First C the next character is i
When the input is First Ci the next character is t
--- Encoded ---
When the input is tensor([18]) the next character is 47
When the input is tensor([18, 47]) the next character is 56
When the input is tensor([18, 47, 56]) the next character is 57
When the input is tensor([18, 47, 56, 57]) the next character is 58
When the input is tensor([18, 47, 56, 57, 58]) the next character is 1
When the input is tensor([18, 47, 56, 57, 58,  1]) the next character is 15
When the input is tensor([18, 47, 56, 57, 58,  1, 15]) the next character is 47
When the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the next character is 58


In [73]:
batch_size = 4
def get_batch(split: typing.Union[typing.Literal['train'], typing.Literal['val']]):
    if split == 'train':
        dataset = train_data
    elif split == 'val':
        dataset = val_data
    else:
        raise f'{split} not a recognized split'

    ix = torch.randint(low=0, high=len(dataset)-block_size, size=(batch_size,), device=device)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [74]:
xb, yb = get_batch('train')
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])


In [75]:
for b in range(batch_size):
    print(f'Example {b}')
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'Block {t}: When the input is {context} the next character is {target}')

Example 0
Block 0: When the input is tensor([53]) the next character is 59
Block 1: When the input is tensor([53, 59]) the next character is 6
Block 2: When the input is tensor([53, 59,  6]) the next character is 1
Block 3: When the input is tensor([53, 59,  6,  1]) the next character is 58
Block 4: When the input is tensor([53, 59,  6,  1, 58]) the next character is 56
Block 5: When the input is tensor([53, 59,  6,  1, 58, 56]) the next character is 47
Block 6: When the input is tensor([53, 59,  6,  1, 58, 56, 47]) the next character is 40
Block 7: When the input is tensor([53, 59,  6,  1, 58, 56, 47, 40]) the next character is 59
Example 1
Block 0: When the input is tensor([49]) the next character is 43
Block 1: When the input is tensor([49, 43]) the next character is 43
Block 2: When the input is tensor([49, 43, 43]) the next character is 54
Block 3: When the input is tensor([49, 43, 43, 54]) the next character is 1
Block 4: When the input is tensor([49, 43, 43, 54,  1]) the next ch

In [ ]:
class BigramLanguageModel(nn.Module):

    token_embedding_table: nn.Embedding

    def __init__(self, vocab_size: int):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx: Tensor, targets: Tensor) -> typing.Tuple[Tensor, Tensor]:
        # idx and targets are (B,T) tensors (batch size by time).
        # logits are (B,T,C) tensors, (batch size by time by channel), where the
        # channel dimension comes from the embedding table.
        logits = self.token_embedding_table(idx)

        B, T, C = logits.shape

        logits = logits.view(B * T, C)
        targets = targets.view(B * T)

        loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx: Tensor, max_new_tokens: int):
        # idx is (B,T,C)
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]

In [82]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)

torch.Size([32, 65])


In [84]:
print(loss)

tensor(4.8354, grad_fn=<NllLossBackward0>)
